# Workshop 02

In [8]:
import cv2
import numpy as np

# Load YOLOv4 network
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")

# Load the COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Set up video capture for webcam
cap = cv2.VideoCapture(0)

last_frame = None  # store final frame for display

while True:
    ret, frame = cap.read()
    if not ret:
        break

    last_frame = frame.copy()   # save frame in case of stopping

    height, width, channels = frame.shape
    object_detected = False     # Reset every frame

    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                object_detected = True

                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]

            color = (0, 255, 0)
            cv2.rectangle(last_frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(last_frame, f"{label} {confidence:.2f}", 
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show live detection until stopped
    cv2.imshow("YOLOv4 Real-Time Object Detection", frame)

    if object_detected:-
        print("Object detected! Stopping detection...")
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Stop detection but KEEP the window open
cap.release()

# Show final frame with detected object
cv2.imshow("Detected Object (Final Frame)", last_frame)
print("Press any key to close window...")
cv2.waitKey(0)

cv2.destroyAllWindows()


Object detected! Stopping detection...
Press any key to close window...


In [ ]:
image = cv2.imread('de')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 